In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import time

from IPython.display import display, clear_output

In [ ]:
class Timer:
    def __init__(self, step):
        self.__time = time.time()
        self.__step = step
    
    def tick(self, speedup = 1):
        elapsed = time.time() - self.__time
        
        wait = self.__step / speedup - elapsed
        workload = (self.__step - wait) / self.__step
        
        if wait >= 0:
            time.sleep(wait)

        self.__time = time.time()
        return workload

In [ ]:
class Aircraft:
    def __init__(self, x, y, spd, rocd):
        self.__x = x
        self.__y = y
        self.__spd = spd
        self.__rocd = rocd
    
    def move(self, step):
        self.__x -= step * self.__spd
        self.__y -= step * self.__rocd
        
    def pos(self):
        return (self.__x, self.__y)

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax_alt_act = fig.add_subplot(2, 2, 1) 
ax_alt_est = fig.add_subplot(2, 2, 2) 
ax_spd_act = fig.add_subplot(2, 2, 3) 
ax_spd_est = fig.add_subplot(2, 2, 4) 

step = .5
timer = Timer(step)

workload = []

ac = Aircraft(15000, 1200, 80, 7)

while True:
    ac.move(step)
    
    (x,y) = ac.pos()
    if x < 0 or y < 0:
        break
        
    ax_alt_act.set_xlim(0, 10)
    ax_alt_act.set_ylim(0, 5000)
    ax_alt_act.plot(x / 1852, y / 0.3048, 'r+')
    
    
    display(fig)
    clear_output(wait = True)
    
    wl = timer.tick(10)
    workload.append(wl)
    
np.mean(workload)